# Notebook 04: Regularization Experiments

**Objective**: Systematically test regularization techniques (Dropout, L2, Batch Normalization) to reduce overfitting and improve generalization.

**Base Architecture**: [128, 64, 32] (selected from Notebook 03 as a balanced medium-depth network)

**Experiments**:
1. **Dropout**: Test rates [0.0, 0.2, 0.3, 0.4, 0.5]
2. **L2 Regularization**: Test strengths [0.0, 0.001, 0.01, 0.1]
3. **Batch Normalization**: With and without
4. **Combined**: Best dropout + best L2 + batch norm

**Evaluation**: Compare validation loss trajectories and PR-AUC to identify optimal regularization strategy.

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import joblib
import sys
import os
import time

# Add src to path
sys.path.append(os.path.abspath('../'))
import config
from src.nn_architectures import create_mlp
from src.nn_training_utils import train_neural_network, get_class_weights
from src.evaluation_metrics import compute_fraud_metrics
from src.visualization_utils import plot_learning_curves

# Set random seeds
config.set_random_seeds()

print("✓ Imports complete")
print(f"✓ TensorFlow version: {tf.__version__}")
print(f"✓ Random seed set to {config.RANDOM_SEED}")

## 1. Load Preprocessed Data

In [ ]:
# Load original data
df = pd.read_csv(config.DATASET_PATH)
X = df[config.FEATURE_COLUMNS].values
y = df[config.TARGET_COLUMN].values

# Load saved split indices
train_idx = np.load(config.TRAIN_INDICES_PATH)
val_idx = np.load(config.VAL_INDICES_PATH)
test_idx = np.load(config.TEST_INDICES_PATH)

# Split data
X_train, y_train = X[train_idx], y[train_idx]
X_val, y_val = X[val_idx], y[val_idx]
X_test, y_test = X[test_idx], y[test_idx]

# Load and apply scaler
scaler = joblib.load(config.FITTED_SCALER_PATH)
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Compute class weights
class_weights = get_class_weights(y_train)

print("✓ Data loaded and preprocessed")
print(f"  Train: {X_train_scaled.shape[0]:,} samples")
print(f"  Val:   {X_val_scaled.shape[0]:,} samples")
print(f"  Test:  {X_test_scaled.shape[0]:,} samples")
print(f"✓ Class weights: {{0: {class_weights[0]:.4f}, 1: {class_weights[1]:.4f}}}")

## 2. Experiment 1: Dropout Regularization

Test dropout rates from 0.0 to 0.5 on base architecture [128, 64, 32].

In [ ]:
# Dropout experiments
base_architecture = [128, 64, 32]
dropout_rates = [0.0, 0.2, 0.3, 0.4, 0.5]
dropout_results = []
dropout_histories = {}

print("="*70)
print("EXPERIMENT 1: DROPOUT REGULARIZATION")
print("="*70)

for dropout in dropout_rates:
    print(f"\n{'='*70}")
    print(f"Training with Dropout = {dropout}")
    print(f"{'='*70}")
    
    start_time = time.time()
    
    # Create model with dropout
    model = create_mlp(
        input_dim=X_train_scaled.shape[1],
        hidden_layers=base_architecture,
        dropout_rate=dropout,
        l2_reg=0.0,
        use_batch_norm=False
    )
    
    # Train
    history, trained_model, train_time = train_neural_network(
        model=model,
        X_train=X_train_scaled,
        y_train=y_train,
        X_val=X_val_scaled,
        y_val=y_val,
        class_weights=class_weights,
        epochs=100,
        batch_size=64,
        patience=15,
        experiment_name=f"dropout_{dropout}"
    )
    
    # Evaluate
    y_val_pred_proba = trained_model.predict(X_val_scaled, verbose=0).flatten()
    y_val_pred = (y_val_pred_proba >= 0.5).astype(int)
    metrics = compute_fraud_metrics(y_val, y_val_pred, y_val_pred_proba)
    
    dropout_results.append({
        'dropout': dropout,
        'pr_auc': metrics['pr_auc'],
        'roc_auc': metrics['roc_auc'],
        'f1_fraud': metrics['fraud_f1'],
        'val_loss': min(history.history['val_loss']),
        'epochs': len(history.history['loss'])
    })
    
    dropout_histories[dropout] = history
    
    print(f"✓ PR-AUC: {metrics['pr_auc']:.4f} | Val Loss: {dropout_results[-1]['val_loss']:.4f}")

print("\n✓ Dropout experiments complete!")

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Dropout comparison
ax1 = axes[0, 0]
dropout_df_plot = dropout_df.sort_values('dropout')
ax1.plot(dropout_df_plot['dropout'], dropout_df_plot['pr_auc'], marker='o', linewidth=2, markersize=8, color='steelblue')
ax1.set_xlabel('Dropout Rate', fontsize=12, fontweight='bold')
ax1.set_ylabel('PR-AUC (Validation)', fontsize=12, fontweight='bold')
ax1.set_title('Dropout Regularization Effect', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.axvline(best_dropout, color='red', linestyle='--', linewidth=2, label=f'Best: {best_dropout}')
ax1.legend()

# 2. L2 comparison
ax2 = axes[0, 1]
l2_df_plot = l2_df.sort_values('l2_reg')
ax2.semilogx(l2_df_plot['l2_reg'][1:], l2_df_plot['pr_auc'][1:], marker='o', linewidth=2, markersize=8, color='coral')
ax2.scatter([l2_df_plot.iloc[0]['l2_reg']], [l2_df_plot.iloc[0]['pr_auc']], s=100, color='coral', marker='o', zorder=5)
ax2.set_xlabel('L2 Regularization Strength', fontsize=12, fontweight='bold')
ax2.set_ylabel('PR-AUC (Validation)', fontsize=12, fontweight='bold')
ax2.set_title('L2 Regularization Effect', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.axvline(best_l2 if best_l2 > 0 else 0.0001, color='red', linestyle='--', linewidth=2, label=f'Best: {best_l2}')
ax2.legend()

# 3. Batch Normalization comparison
ax3 = axes[1, 0]
bn_df_plot = bn_df
colors = ['#e74c3c' if row['batch_norm'] == 'Yes' and best_bn else '#3498db' if row['batch_norm'] == 'No' and not best_bn else '#95a5a6' for _, row in bn_df_plot.iterrows()]
bars = ax3.bar(range(len(bn_df_plot)), bn_df_plot['pr_auc'], color=colors, edgecolor='black', linewidth=2)
ax3.set_xticks(range(len(bn_df_plot)))
ax3.set_xticklabels(bn_df_plot['batch_norm'])
ax3.set_xlabel('Batch Normalization', fontsize=12, fontweight='bold')
ax3.set_ylabel('PR-AUC (Validation)', fontsize=12, fontweight='bold')
ax3.set_title('Batch Normalization Effect', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3, axis='y')

# 4. All techniques comparison
ax4 = axes[1, 1]
all_results = {
    'Baseline\\n(no reg)': dropout_df[dropout_df['dropout'] == 0.0]['pr_auc'].values[0],
    f'Best Dropout\\n({best_dropout})': dropout_df[dropout_df['dropout'] == best_dropout]['pr_auc'].values[0],
    f'Best L2\\n({best_l2})': l2_df[l2_df['l2_reg'] == best_l2]['pr_auc'].values[0],
    f'Best BN\\n({\"Yes\" if best_bn else \"No\"})': bn_df[bn_df['batch_norm'] == ('Yes' if best_bn else 'No')]['pr_auc'].values[0],
    'Combined': metrics_combined['pr_auc']
}
bars = ax4.bar(range(len(all_results)), list(all_results.values()), color='purple', alpha=0.7, edgecolor='black', linewidth=2)
bars[-1].set_color('#2ecc71')  # Highlight combined
ax4.set_xticks(range(len(all_results)))
ax4.set_xticklabels(list(all_results.keys()), rotation=0, fontsize=10)
ax4.set_ylabel('PR-AUC (Validation)', fontsize=12, fontweight='bold')
ax4.set_title('Regularization Techniques Comparison', fontsize=13, fontweight='bold')
ax4.grid(True, alpha=0.3, axis='y')

for i, (label, value) in enumerate(all_results.items()):
    ax4.text(i, value + 0.005, f'{value:.4f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.suptitle('Regularization Experiments - Fraud Detection Neural Network', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig('../results/figures/regularization_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Save all results
all_reg_results = pd.concat([
    dropout_df.assign(experiment='dropout'),
    l2_df.assign(experiment='l2'),
    bn_df.assign(experiment='batch_norm')
], ignore_index=True)

os.makedirs('../results/tables', exist_ok=True)
all_reg_results.to_csv('../results/tables/regularization_experiments.csv', index=False)

print("\n" + "="*70)
print(" NOTEBOOK 04 SUMMARY - REGULARIZATION EXPERIMENTS")
print("="*70)
print(f"✓ Dropout experiments: {len(dropout_rates)} configurations tested")
print(f"✓ L2 experiments: {len(l2_strengths)} configurations tested")
print(f"✓ Batch Norm experiments: 2 configurations tested")
print(f"\n🏆 Best Hyperparameters:")
print(f"  Dropout: {best_dropout}")
print(f"  L2: {best_l2}")
print(f"  Batch Normalization: {'Yes' if best_bn else 'No'}")
print(f"\n📊 Performance:")
print(f"  Baseline (no reg): {all_results['Baseline\\n(no reg)']:.4f}")
print(f"  Combined optimal: {metrics_combined['pr_auc']:.4f}")
print(f"  Improvement: +{(metrics_combined['pr_auc'] - all_results['Baseline\\n(no reg)']) / all_results['Baseline\\n(no reg)'] * 100:.2f}%")
print("\n✅ Notebook 04 Complete!")
print("🚀 Ready for Notebook 05: Class Imbalance Strategies")
print("="*70)

## 6. Visualization & Summary

In [ ]:
# Find best hyperparameters from each experiment
dropout_df = pd.DataFrame(dropout_results)
l2_df = pd.DataFrame(l2_results)
bn_df = pd.DataFrame(bn_results)

best_dropout = dropout_df.loc[dropout_df['pr_auc'].idxmax(), 'dropout']
best_l2 = l2_df.loc[l2_df['pr_auc'].idxmax(), 'l2_reg']
best_bn = bn_df.loc[bn_df['pr_auc'].idxmax(), 'batch_norm'] == 'Yes'

print("="*70)
print("OPTIMAL HYPERPARAMETERS")
print("="*70)
print(f"Best Dropout: {best_dropout}")
print(f"Best L2: {best_l2}")
print(f"Best Batch Norm: {'Yes' if best_bn else 'No'}")

# Train combined model
print(f"\n{'='*70}")
print("Training COMBINED model with optimal settings")
print(f"{'='*70}")

model_combined = create_mlp(
    input_dim=X_train_scaled.shape[1],
    hidden_layers=base_architecture,
    dropout_rate=best_dropout,
    l2_reg=best_l2,
    use_batch_norm=best_bn
)

history_combined, model_combined_trained, _ = train_neural_network(
    model=model_combined,
    X_train=X_train_scaled,
    y_train=y_train,
    X_val=X_val_scaled,
    y_val=y_val,
    class_weights=class_weights,
    epochs=100,
    batch_size=64,
    patience=15,
    experiment_name="combined_regularization"
)

# Evaluate
y_val_pred_proba = model_combined_trained.predict(X_val_scaled, verbose=0).flatten()
y_val_pred = (y_val_pred_proba >= 0.5).astype(int)
metrics_combined = compute_fraud_metrics(y_val, y_val_pred, y_val_pred_proba)

print(f"\n✓ Combined model PR-AUC: {metrics_combined['pr_auc']:.4f}")
print(f"✓ Combined model saved to: ../models/combined_regularization.keras")
model_combined_trained.save('../models/combined_regularization.keras')

## 5. Combined Regularization

Train model with optimal combination of regularization techniques.

In [ ]:
# Batch Normalization experiments
bn_configs = [False, True]
bn_results = []
bn_histories = {}

print("="*70)
print("EXPERIMENT 3: BATCH NORMALIZATION")
print("="*70)

for use_bn in bn_configs:
    print(f"\n{'='*70}")
    print(f"Training {'WITH' if use_bn else 'WITHOUT'} Batch Normalization")
    print(f"{'='*70}")
    
    # Create model
    model = create_mlp(
        input_dim=X_train_scaled.shape[1],
        hidden_layers=base_architecture,
        dropout_rate=0.0,
        l2_reg=0.0,
        use_batch_norm=use_bn
    )
    
    # Train
    history, trained_model, train_time = train_neural_network(
        model=model,
        X_train=X_train_scaled,
        y_train=y_train,
        X_val=X_val_scaled,
        y_val=y_val,
        class_weights=class_weights,
        epochs=100,
        batch_size=64,
        patience=15,
        experiment_name=f"batchnorm_{use_bn}"
    )
    
    # Evaluate
    y_val_pred_proba = trained_model.predict(X_val_scaled, verbose=0).flatten()
    y_val_pred = (y_val_pred_proba >= 0.5).astype(int)
    metrics = compute_fraud_metrics(y_val, y_val_pred, y_val_pred_proba)
    
    bn_results.append({
        'batch_norm': 'Yes' if use_bn else 'No',
        'pr_auc': metrics['pr_auc'],
        'roc_auc': metrics['roc_auc'],
        'f1_fraud': metrics['fraud_f1'],
        'val_loss': min(history.history['val_loss']),
        'epochs': len(history.history['loss'])
    })
    
    bn_histories[use_bn] = history
    
    print(f"✓ PR-AUC: {metrics['pr_auc']:.4f} | Val Loss: {bn_results[-1]['val_loss']:.4f}")

print("\n✓ Batch Normalization experiments complete!")

## 4. Experiment 3: Batch Normalization

Compare performance with and without Batch Normalization.

In [ ]:
# L2 regularization experiments
l2_strengths = [0.0, 0.001, 0.01, 0.1]
l2_results = []
l2_histories = {}

print("="*70)
print("EXPERIMENT 2: L2 REGULARIZATION")
print("="*70)

for l2_reg in l2_strengths:
    print(f"\n{'='*70}")
    print(f"Training with L2 = {l2_reg}")
    print(f"{'='*70}")
    
    # Create model with L2
    model = create_mlp(
        input_dim=X_train_scaled.shape[1],
        hidden_layers=base_architecture,
        dropout_rate=0.0,
        l2_reg=l2_reg,
        use_batch_norm=False
    )
    
    # Train
    history, trained_model, train_time = train_neural_network(
        model=model,
        X_train=X_train_scaled,
        y_train=y_train,
        X_val=X_val_scaled,
        y_val=y_val,
        class_weights=class_weights,
        epochs=100,
        batch_size=64,
        patience=15,
        experiment_name=f"l2_{l2_reg}"
    )
    
    # Evaluate
    y_val_pred_proba = trained_model.predict(X_val_scaled, verbose=0).flatten()
    y_val_pred = (y_val_pred_proba >= 0.5).astype(int)
    metrics = compute_fraud_metrics(y_val, y_val_pred, y_val_pred_proba)
    
    l2_results.append({
        'l2_reg': l2_reg,
        'pr_auc': metrics['pr_auc'],
        'roc_auc': metrics['roc_auc'],
        'f1_fraud': metrics['fraud_f1'],
        'val_loss': min(history.history['val_loss']),
        'epochs': len(history.history['loss'])
    })
    
    l2_histories[l2_reg] = history
    
    print(f"✓ PR-AUC: {metrics['pr_auc']:.4f} | Val Loss: {l2_results[-1]['val_loss']:.4f}")

print("\n✓ L2 experiments complete!")

## 3. Experiment 2: L2 Regularization

Test L2 penalty strengths from 0.0 to 0.1 on base architecture.